In [1]:
!pip install -q -U transformers datasets accelerate peft trl bitsandbytes scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
import os
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer

from huggingface_hub import notebook_login
notebook_login()

In [3]:
model_id = "google/gemma-3-12b-it"

new_model_name = "gemma-3-12b-it-medium-json"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
eos_token = tokenizer.eos_token

print(f"EOS Token: {eos_token}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

EOS Token: <eos>


In [ ]:
print(f"Sample Data Point (check EOS):\n{train_dataset[0]['formatted_text']}")

Sample Data Point (check EOS):
Convert the following food order into JSON format using this structure:
[
  { "customizations": ["Customization 1", "Customization 2"], "name": "Item Name" },
  { "customizations": [], "name": "Another Item" }
]:
Miso soup with toasted sesame seeds and chili oil, extra crispy fish and chips with fries and onion rings, chocolate cake, apple pie, and lemon meringue pie.
[{"customizations":["Toasted Sesame Seeds","Chili Oil"],"name":"Miso Soup"},{"customizations":["Extra Crispy","Fries","Onion Rings","Crispy"],"name":"Fish and Chips"},{"customizations":[],"name":"Chocolate Cake"},{"customizations":[],"name":"Apple Pie"},{"customizations":[],"name":"Lemon Meringue Pie"}]<eos>


In [ ]:
import pandas as pd

df = train_dataset.to_pandas()
num_duplicates = df.duplicated().sum()
num_duplicates

np.int64(0)

In [5]:
from datasets import load_dataset
import json

dataset_id = "iTzMiNOS/voice-orders-medium-clean-18k"
split_name = "train"

print(f"Loading dataset '{dataset_id}' (split: '{split_name}')...")
dataset = load_dataset(dataset_id, split=split_name)

columns_to_keep = ["transcribed_text", "items", "speaker"]
dataset = dataset.remove_columns([col for col in dataset.column_names if col not in columns_to_keep])

dataset = dataset.select(range(min(1800, len(dataset))))

print(f"Dataset loaded: {dataset}")

prefix = """Convert the following food order into JSON format using this structure:
[
  { "customizations": ["Customization 1", "Customization 2"], "name": "Item Name" },
  { "customizations": [], "name": "Another Item" }
]:
"""

print("Splitting dataset into train and validation sets...")
train_val_split = dataset.train_test_split(test_size=0.1, shuffle=True, seed=42)
train_data = train_val_split['train']
validation_data = train_val_split['test']

print(f"Train set size: {len(train_data)}")
print(f"Validation set size: {len(validation_data)}")

def format_data_for_sft(example):
    text = example["transcribed_text"]
    items_data = example["items"]
    json_string = json.dumps(items_data, separators=(',', ':'))
    if 'tokenizer' not in globals():
        raise NameError("Tokenizer not found. Please run the tokenizer loading cell (Cell 5) first.")
    eos = tokenizer.eos_token
    formatted_string = f"{prefix}{text}\n{json_string}{eos}"
    return {"formatted_text": formatted_string}

if 'tokenizer' in globals():
    print("Applying formatting function to the datasets...")
    train_dataset = train_data.map(format_data_for_sft, remove_columns=train_data.column_names)
    validation_dataset = validation_data.map(format_data_for_sft, remove_columns=validation_data.column_names) # Process validation data too!
    print("Dataset formatting complete.")
    print(f"Train dataset features: {train_dataset.features}")
    print(f"Validation dataset features: {validation_dataset.features}")
else:
    print("WARNING: Tokenizer not loaded yet. Re-run Cell 5 and this cell's mapping part.")
    train_dataset = None
    validation_dataset = None

Loading dataset 'iTzMiNOS/voice-orders-medium-clean-18k' (split: 'train')...


README.md:   0%|          | 0.00/518 [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/28 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/28 [00:00<?, ?it/s]

train-00000-of-00028.parquet:   0%|          | 0.00/487M [00:00<?, ?B/s]

...

Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

Dataset loaded: Dataset({
    features: ['transcribed_text', 'speaker', 'items'],
    num_rows: 1800
})
Splitting dataset into train and validation sets...
Train set size: 1620
Validation set size: 180
Applying formatting function to the datasets...


Map:   0%|          | 0/1620 [00:00<?, ? examples/s]

Map:   0%|          | 0/180 [00:00<?, ? examples/s]

Dataset formatting complete.
Train dataset features: {'formatted_text': Value(dtype='string', id=None)}
Validation dataset features: {'formatted_text': Value(dtype='string', id=None)}


In [6]:
validation_data

Dataset({
    features: ['transcribed_text', 'speaker', 'items'],
    num_rows: 180
})

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

if torch.cuda.is_available() and torch.cuda.get_device_capability()[0] >= 8:
    print("GPU supports bfloat16, using torch.bfloat16")
    compute_dtype = torch.bfloat16
else:
    print("GPU does not support bfloat16, using torch.float16")
    compute_dtype = torch.float16
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=False,
    )

print("BitsAndBytesConfig configured.")

GPU supports bfloat16, using torch.bfloat16
BitsAndBytesConfig configured.


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
)

model = prepare_model_for_kbit_training(model)

print("Base model loaded and prepared for k-bit training.")
print(model)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Base model loaded and prepared for k-bit training.
Gemma3ForConditionalGeneration(
  (vision_tower): SiglipVisionModel(
    (vision_model): SiglipVisionTransformer(
      (embeddings): SiglipVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
        (position_embedding): Embedding(4096, 1152)
      )
      (encoder): SiglipEncoder(
        (layers): ModuleList(
          (0-26): 27 x SiglipEncoderLayer(
            (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
            (self_attn): SiglipAttention(
              (k_proj): Linear4bit(in_features=1152, out_features=1152, bias=True)
              (v_proj): Linear4bit(in_features=1152, out_features=1152, bias=True)
              (q_proj): Linear4bit(in_features=1152, out_features=1152, bias=True)
              (out_proj): Linear4bit(in_features=1152, out_features=1152, bias=True)
            )
            (layer_norm2): LayerNorm((1152,), eps=1e-06, 

In [ ]:
peft_config = LoraConfig(
    lora_alpha=128,
    lora_dropout=0.05,
    r=16,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
)

In [ ]:
training_arguments = TrainingArguments(
    output_dir=new_model_name,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    eval_strategy="steps",
    eval_steps=50,
    logging_steps=10,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=(compute_dtype==torch.float16),
    bf16=(compute_dtype==torch.bfloat16),
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="tensorboard",
    save_strategy="steps",
    save_steps=50,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)


from trl import SFTConfig

sft_config = SFTConfig(
    output_dir=new_model_name,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    eval_strategy="steps",
    eval_steps=50,
    logging_steps=10,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=(compute_dtype==torch.float16),
    bf16=(compute_dtype==torch.bfloat16),
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="tensorboard",
    save_strategy="steps",
    save_steps=50,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    max_seq_length=1024,
    packing=False,
    dataset_text_field="formatted_text",
    push_to_hub=True,
    hub_model_id=f"iTzMiNOS/{new_model_name}",
)


print("SFTConfig configured.")



SFTConfig configured.


In [ ]:
if 'train_dataset' not in globals() or 'validation_dataset' not in globals() or train_dataset is None or validation_dataset is None:
     raise ValueError("Training or Validation dataset not found or not processed. Please ensure Cell 4 ran correctly after loading the tokenizer.")
if 'model' not in globals():
      raise ValueError("Model not loaded.")
if 'tokenizer' not in globals():
      raise ValueError("Tokenizer not loaded.")
if 'sft_config' not in globals():
      raise ValueError("SFTConfig not defined.")
if 'peft_config' not in globals():
      raise ValueError("peft_config not defined.")

tokenizer.chat_template = None
print("Attempting SFTTrainer initialization using SFTConfig...")
try:
    trainer = SFTTrainer(
        model=model,
        args=sft_config,
        train_dataset=train_dataset,
        eval_dataset=validation_dataset,
        peft_config=peft_config,
        processing_class=tokenizer,
    )
    print("SFTTrainer initialized successfully using SFTConfig!")

except TypeError as e:
    print(f"SFTTrainer STILL FAILED with TypeError: {e}")
    print("Check if 'tokenizer' is now the unexpected argument.")
except Exception as e:
    print(f"SFTTrainer FAILED with another error: {e}")

Attempting SFTTrainer initialization using SFTConfig...


Converting train dataset to ChatML:   0%|          | 0/1620 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/1620 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1620 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1620 [00:00<?, ? examples/s]

...

Adding EOS to eval dataset:   0%|          | 0/180 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/180 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/180 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


SFTTrainer initialized successfully using SFTConfig!


In [ ]:
print("Starting fine-tuning...")
trainer.train()
print("Fine-tuning finished.")

Starting fine-tuning...


It is strongly recommended to train Gemma3 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss
50,1.054400,0.321484
100,0.983900,0.331645
150,0.937100,0.299923
200,0.892600,0.296216
250,0.816400,0.275030
300,0.811100,0.258903
350,0.848800,0.245400
400,0.797700,0.245594
450,0.707800,0.243390
500,0.681500,0.238000


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

Fine-tuning finished.


In [ ]:
print(f"Saving fine-tuned adapters to ./{new_model_name}")
trainer.model.save_pretrained(new_model_name)
tokenizer.save_pretrained(new_model_name)
print("Adapters and tokenizer saved.")

# Optional: Clean up memory
# del model
# del trainer
# torch.cuda.empty_cache()

Saving fine-tuned adapters to ./gemma-3-12b-it-medium-json
Adapters and tokenizer saved.


In [7]:
# Cell: Inference and Evaluation (Batched & GPU Ensured & Fence Cleaning)

import json
import pandas as pd
from datasets import Dataset
from transformers import (
    pipeline,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    logging
)
from peft import PeftModel
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from difflib import SequenceMatcher
import torch
import gc
import os
import math
import numpy as np
import re # Import regex for cleaning fences

# --- Configuration ---
base_model_id = "google/gemma-3-12b-it"
adapter_model_id = "iTzMiNOS/gemma-3-12b-it-medium-json"
prefix = """Convert the following food order into JSON format using this structure:
[
  { "customizations": ["Customization 1", "Customization 2"], "name": "Item Name" },
  { "customizations": [], "name": "Another Item" }
]:
"""
inference_batch_size = 16

# --- Check GPU Availability & L4 ---
if not torch.cuda.is_available():
    raise SystemError("CUDA is not available. This script requires a GPU.")
else:
    device_name = torch.cuda.get_device_name(0)
    print(f"CUDA is available. Using GPU: {device_name}")
    if "L4" not in device_name:
         print("Warning: The detected GPU is not an L4. Performance may vary.")
    device = 0

# --- Memory Cleanup ---
print("Cleaning up memory before loading...")
gc.collect()
torch.cuda.empty_cache()
print("CUDA cache cleared.")

# --- Setup Quantization ---
compute_dtype = torch.bfloat16 if torch.cuda.get_device_capability(device)[0] >= 8 else torch.float16
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)
print(f"Using compute dtype: {compute_dtype} for BnB config.")

# --- Load Tokenizer ---
print(f"Loading tokenizer from {base_model_id}...")
tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"
eos_token = tokenizer.eos_token
assert tokenizer.pad_token_id is not None, "Tokenizer pad_token_id is not set!"
print("Tokenizer loaded.")

# --- Load Base Model with Quantization ---
print(f"Loading base model ({base_model_id}) with quantization...")
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=compute_dtype,
    trust_remote_code=True,
)
print(f"Base model loaded. Device map: {model.hf_device_map}")

# --- Load LoRA Adapter ---
print(f"Loading LoRA adapter ({adapter_model_id}) onto the base model...")
try:
    model = PeftModel.from_pretrained(model, adapter_model_id)
    print("LoRA adapter loaded successfully.")
    print("Attempting to merge LoRA adapter...")
    try:
        model = model.merge_and_unload()
        print("LoRA adapter merged successfully.")
    except Exception as e:
        print(f"⚠️ Could not merge LoRA adapter: {e}. Proceeding with PEFT model.")
except Exception as e:
     print(f"❌ Failed to load LoRA adapter: {e}")
     raise e

# --- Build the Inference Pipeline ---
logging.set_verbosity(logging.CRITICAL)
print("Building text-generation pipeline...")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)
if hasattr(pipe, 'device'): print(f"Pipeline device: {pipe.device}")
else: print("Pipeline device managed by model's device_map.")

# --- Load Validation Data ---
if 'validation_data' not in globals():
     print("validation_data not found, attempting reload...")
     from datasets import load_dataset
     dataset_id = "iTzMiNOS/voice-orders-medium-clean-18k"
     split_name = "train"
     dataset = load_dataset(dataset_id, split=split_name)
     columns_to_keep = ["transcribed_text", "items", "speaker"]
     dataset = dataset.remove_columns([col for col in dataset.column_names if col not in columns_to_keep])
     dataset = dataset.select(range(min(1200, len(dataset))))
     train_val_split = dataset.train_test_split(test_size=0.1, shuffle=True, seed=42)
     validation_data = train_val_split['test']
     print("Reloaded and split dataset.")

df = validation_data.to_pandas()
print(f"Loaded validation data with {len(df)} rows.")

# --- Convert Numpy arrays in 'items' column ---
def deep_convert(obj):
    if isinstance(obj, dict):
        return {k: deep_convert(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [deep_convert(v) for v in obj]
    elif isinstance(obj, np.ndarray):
        return deep_convert(obj.tolist())
    else:
        return obj

df['items'] = df['items'].apply(deep_convert)
print("Conversion complete.")


# --- ***** UPDATED: Function for Robust JSON Parsing (Handles Fences) ***** ---
def parse_json_robustly(generated_text):
    """Attempts to extract and parse JSON, handling optional markdown fences."""
    json_str = None
    try:
        # Find the start of the first list or object
        first_bracket = generated_text.find('[')
        first_brace = generated_text.find('{')

        start_index = -1
        if first_bracket != -1 and (first_brace == -1 or first_bracket < first_brace):
            start_index = first_bracket
            start_char = '['
            end_char = ']'
        elif first_brace != -1:
            start_index = first_brace
            start_char = '{'
            end_char = '}'
        else:
             # Neither bracket nor brace found - maybe it's ONLY fences?
             # Try finding fences directly if no brace/bracket
             fence_match = re.search(r"`{3}(json)?\s*([\[\{])", generated_text)
             if fence_match:
                 start_index = fence_match.end() -1 # Start at the brace/bracket
                 start_char = fence_match.group(2)
                 end_char = ']' if start_char == '[' else '}'
             else:
                # Give up if no structure found
                # print(f"Debug: No JSON start ('[' or '{{') found. Output: {generated_text}")
                return None

        # Find the corresponding closing character using balancing
        open_count = 0
        end_index = -1
        # Check if start_index is valid before proceeding
        if start_index >= 0 and start_index < len(generated_text):
            for i in range(start_index, len(generated_text)):
                if generated_text[i] == start_char:
                    open_count += 1
                elif generated_text[i] == end_char:
                    open_count -= 1
                if open_count == 0:
                    end_index = i
                    break
        else:
             # Handle invalid start_index if fence logic above failed unusually
             print(f"Debug: Invalid start_index {start_index}. Output: {generated_text}")
             return None


        if end_index == -1:
            # print(f"Debug: No matching closing bracket/brace. Output: {generated_text}")
            return None

        # Extract the potential JSON substring
        json_str = generated_text[start_index : end_index + 1]

        # Clean leading/trailing whitespace that might remain
        json_str = json_str.strip()

        # --- Attempt to parse the extracted & cleaned string ---
        json_data = json.loads(json_str)
        return json_data

    except json.JSONDecodeError as e:
        # Add logging for parse failures, include the string attempted
        print(f"Warning: Could not parse JSON: {e}.")
        print(f"Attempted to parse (after extraction): '{json_str}'")
        # print(f"Original Generated Text: {generated_text}") # Uncomment for deeper debugging
        return None
    except Exception as e:
        # Catch any other unexpected errors during parsing
        print(f"Warning: Unexpected error during JSON parsing: {e}.")
        print(f"Attempted to parse (after extraction): '{json_str}'")
        # print(f"Original Generated Text: {generated_text}") # Uncomment for deeper debugging
        return None
# --- End Updated Function ---


# --- Apply Inference with Batching (Unchanged) ---
print(f"Running batched inference (batch size: {inference_batch_size})...")
all_prompts = [f"{prefix}{text}" for text in df['transcribed_text']]
all_results = []
num_batches = math.ceil(len(all_prompts) / inference_batch_size)
for i in tqdm(range(0, len(all_prompts), inference_batch_size), desc="Inference Batches", total=num_batches):
    batch_prompts = all_prompts[i:i+inference_batch_size]
    try:
        batch_outputs = pipe(batch_prompts, max_new_tokens=500, return_full_text=False, pad_token_id=tokenizer.eos_token_id, batch_size=len(batch_prompts))
        for output_list in batch_outputs:
            if output_list and isinstance(output_list, list):
                 generated_text = output_list[0]["generated_text"].strip()
                 parsed_json = parse_json_robustly(generated_text) # Use updated parser
                 all_results.append(parsed_json)
            else: print(f"Warning: Unexpected output format: {output_list}"); all_results.append(None)
    except Exception as e:
        print(f"\n--- ERROR during batch {i // inference_batch_size + 1} --- Error: {e}")
        all_results.extend([None] * len(batch_prompts))
if len(all_results) != len(all_prompts):
     print(f"Warning: Result count mismatch! Padding with None.")
     all_results.extend([None] * (len(all_prompts) - len(all_results)))
df['predicted_items'] = all_results
print("Inference complete.")

def to_lower(obj):
    if isinstance(obj, str):
        return obj.lower()  # Convert strings to lowercase
    elif isinstance(obj, dict):
        return {k: to_lower(v) for k, v in obj.items()}  # Apply recursively for dictionaries
    elif isinstance(obj, list):
        return [to_lower(v) for v in obj]  # Apply recursively for lists
    else:
        return obj

# --- Comparison Metric (Unchanged) ---
def similarity_score(pred, target):
    if pred is None or target is None:
        return 0.0
    try:
        # Convert both the prediction and target to lowercase
        pred = to_lower(pred)
        target = to_lower(target)

        # Convert the structures into strings
        pred_str = json.dumps(pred, sort_keys=True, separators=(',', ':'))
        target_str = json.dumps(target, sort_keys=True, separators=(',', ':'))

        return SequenceMatcher(None, pred_str, target_str).ratio()
    except Exception as e:
        print(f"Error calculating similarity: Pred={pred}, Target={target}, Error={e}")
        return 0.0
# --- Calculate Metrics (Unchanged) ---
print("Calculating metrics...")
df['similarity'] = df.apply(lambda row: similarity_score(row['predicted_items'], row['items']), axis=1)
df['exact_match'] = df.apply(lambda row:
                             row['predicted_items'] is not None and
                             row['items'] is not None and
                             to_lower(row['predicted_items']) == to_lower(row['items']),
                             axis=1)

average_similarity = df['similarity'].mean()
exact_match_accuracy = df['exact_match'].mean()

print("\n--- Evaluation Results ---")
print(f"🔍 Average Similarity Score: {average_similarity:.4f}")
print(f"✅ Exact Match Accuracy: {exact_match_accuracy:.2%}")

# --- Display Mismatches (Unchanged) ---
print("\n--- Low Similarity Examples (< 0.8) ---")
low_sim_df = df[df['similarity'] < 0.8][['transcribed_text', 'items', 'predicted_items', 'similarity']]
print(low_sim_df.to_string())

CUDA is available. Using GPU: NVIDIA L4
Cleaning up memory before loading...
CUDA cache cleared.
Using compute dtype: torch.bfloat16 for BnB config.
Loading tokenizer from google/gemma-3-12b-it...
Tokenizer loaded.
Loading base model (google/gemma-3-12b-it) with quantization...


config.json:   0%|          | 0.00/916 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/109k [00:00<?, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/4.60G [00:00<?, ?B/s]

...

LoRA adapter loaded successfully.
Attempting to merge LoRA adapter...


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:351: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


LoRA adapter merged successfully.
Building text-generation pipeline...
Pipeline device: cuda:0
Loaded validation data with 180 rows.
Conversion complete.
Running batched inference (batch size: 16)...


Inference Batches: 100%|██████████| 12/12 [37:52<00:00, 189.37s/it]

Inference complete.
Calculating metrics...

--- Evaluation Results ---
🔍 Average Similarity Score: 0.9877
✅ Exact Match Accuracy: 74.44%

--- Low Similarity Examples (< 0.8) ---
                                                                                                                                           transcribed_text                                                                                                                                                                                                                                                                                                                      items                                                                                                                                                                                                                                                                                                                                                                

In [8]:
df.head(5)

,transcribed_text,speaker,items,predicted_items,similarity,exact_match
0,I want a Cream of Mushroom soup with truffle o...,af_bella,"[{'customizations': ['Truffle Oil'], 'name': '...","[{'customizations': ['Truffle Oil'], 'name': '...",0.919140,False
1,Ordering some mozzarella sticks with marinara ...,af_bella,"[{'customizations': ['Marinara', 'Ranch'], 'na...","[{'customizations': ['Marinara', 'Ranch'], 'na...",0.991515,False
2,I'll have the Cream of Mushroom with chopped p...,af_bella,"[{'customizations': ['Chopped Parsley', 'Truff...","[{'customizations': ['Chopped Parsley', 'Truff...",0.852417,False
3,"One vegan burger with vegan mayo, a mix of Spr...",af_bella,"[{'customizations': ['Vegan Mayo'], 'name': 'V...","[{'customizations': ['Vegan Mayo'], 'name': 'V...",1.000000,True
4,Ordering a Chicken Soup with fresh parsley and...,af_bella,"[{'customizations': ['Fresh Parsley', 'Lemon W...","[{'customizations': ['Fresh Parsley', 'Lemon W...",1.000000,True


In [ ]:
pipe(prefix + , max_new_tokens=200, return_full_text=False, pad_token_id=tokenizer.eos_token_id)